In [1]:
from sklearn import cross_validation
from scikits.statsmodels.tools import categorical
from sklearn import linear_model
import time
import csv

In [2]:
execfile('ML_Challenge_data_preprocessing.py')

('Generated X with shape ', (347698, 1000))
('Generated y with shape ', (347698,))


In [3]:
bigrams = []
with open('possible_bigrams.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        bigrams = row

bigrams.sort()
bigrams_dict = {key: i for i, key in enumerate(bigrams)}
bigrams_dict

{'AA': 0,
 'AC': 1,
 'AG': 2,
 'AT': 3,
 'CA': 4,
 'CC': 5,
 'CG': 6,
 'CT': 7,
 'GA': 8,
 'GC': 9,
 'GG': 10,
 'GT': 11,
 'TA': 12,
 'TC': 13,
 'TG': 14,
 'TT': 15}

In [14]:
# Remove bad data
X = np.array(filter(lambda x: 'N' not in x, X))
print X.shape

(347683, 1000)


In [32]:
import time
execfile('ngram_features.py')
# for every observation, create a new feature vector or length len(bigrams_dict.keys())

sample_size = 10000#X.shape[0])
num_pair_types = 4
start = time.clock()
newX = np.zeros((sample_size, len(bigrams_dict) + num_pair_types*X.shape[1]))
print newX.shape
for i in range(sample_size):
    one_hot = categorical(X[i], drop = True)
    one_hot = one_hot.flatten()
    ngram_features_x = ngram_features(X[i,:], 2, bigrams_dict)
    newobs = ngram_features_x + list(one_hot)
    newX[i,:] = newobs
finish = time.clock()
print 'time spent creating new features for {0} observations: {1}'.format(sample_size, finish - start)
# will take about 11 minutes for the whole data set
print newX.shape
print newX[0,:]

(10000, 4016)
time spent creating new features for 10000 observations: 29.014793
(10000, 4016)
[ 15.  31.  59. ...,   1.   0.   0.]


In [33]:
test_size = 0.1
y_sample = y[0:sample_size]
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    newX, y_sample, test_size=test_size, random_state=0)
print X_train.shape
print y_train.shape

(9000, 4016)
(9000,)


In [38]:
import time

start_time = time.clock()
logreg = linear_model.LogisticRegression()
logreg.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

time spent to train 0.9: 13.278779 seconds.


In [39]:
logreg.score(X_test, y_test)

0.872

In [ ]:
from sklearn.svm import SVC

start_time = time.clock()
# linear seems best so far achieving up to 0.89
clf = SVC(kernel = 'poly')
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

In [57]:
clf.score(X_test, y_test)

0.89100000000000001

In [40]:
from sklearn.tree import DecisionTreeClassifier

start_time = time.clock()
clf = DecisionTreeClassifier(max_depth=5)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            random_state=None, splitter='best')
time spent to train 0.9: 4.739286 seconds.


In [41]:
clf.score(X_test, y_test)

0.88300000000000001

In [42]:
from sklearn.ensemble import AdaBoostClassifier

start_time = time.clock()
clf = AdaBoostClassifier(n_estimators=25, learning_rate=1)
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1,
          n_estimators=25, random_state=None)
time spent to train 0.9: 20.174767 seconds.


In [43]:
clf.score(X_test, y_test)

0.86199999999999999

In [48]:
from sklearn.ensemble import RandomForestClassifier

start_time = time.clock()
clf = RandomForestClassifier(n_estimators=50, criterion='entropy')
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
time spent to train 0.9: 4.580618 seconds.


In [49]:
clf.score(X_test, y_test)

0.86499999999999999

In [95]:
from sklearn.naive_bayes import GaussianNB

start_time = time.clock()
clf = GaussianNB()
print clf.fit(X_train, y_train)
end_time = time.clock()

print 'time spent to train ' + str(1 - test_size) + ': ' + str(end_time - start_time) + ' seconds.'

GaussianNB()
time spent to train 0.9: 0.00417899999999 seconds.


In [96]:
clf.score(X_test, y_test)

0.86399999999999999

In [100]:
from sklearn import neighbors

max_neighbors=100
inc_by = 3
# starting from 2 - 20 neighbors went approximately linearly from 0.83 to 0.877
num_neighbors = range(3,max_neighbors,inc_by)
accuracies = []
for k in num_neighbors:
    clf = neighbors.KNeighborsClassifier(k)#, weights='distance')
    clf.fit(X_train, y_train)
    start_fit = time.clock()
    score = clf.score(X_test, y_test)
    finish_fit = time.clock()
    accuracies.append(score)    
    print 'Accuracy for ' + str(k) + ' neighbors: ' + str(score)
    print 'Time spent calculating neighbors: ' + str(finish_fit - start_fit)
    print ''

Accuracy for 3 neighbors: 0.897
Time spent calculating neighbors: 0.099258

Accuracy for 6 neighbors: 0.889
Time spent calculating neighbors: 0.08288

Accuracy for 9 neighbors: 0.884
Time spent calculating neighbors: 0.103587

Accuracy for 12 neighbors: 0.881
Time spent calculating neighbors: 0.109193

Accuracy for 15 neighbors: 0.886
Time spent calculating neighbors: 0.1285

Accuracy for 18 neighbors: 0.887
Time spent calculating neighbors: 0.135845

Accuracy for 21 neighbors: 0.888
Time spent calculating neighbors: 0.22269

Accuracy for 24 neighbors: 0.89
Time spent calculating neighbors: 0.15002

Accuracy for 27 neighbors: 0.89
Time spent calculating neighbors: 0.200593

Accuracy for 30 neighbors: 0.89
Time spent calculating neighbors: 0.193924

Accuracy for 33 neighbors: 0.891
Time spent calculating neighbors: 0.13983

Accuracy for 36 neighbors: 0.894
Time spent calculating neighbors: 0.137444

Accuracy for 39 neighbors: 0.891
Time spent calculating neighbors: 0.143478

Accuracy fo